In [1]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import pickle

# Part 0 : Importing and Normalizing Data

In [2]:
transform = transforms.Compose([transforms.ToTensor()])

In [3]:
trainset = datasets.FashionMNIST('./F_MNIST_data/'
                                 , train = True, transform=transform, download=True)

In [4]:
trainloader = torch.utils.data.DataLoader(trainset,  shuffle=True, batch_size=60)

In [5]:
testset = datasets.FashionMNIST('./F_MNIST_data/',
                               train=False, transform=transform,
                               download=True)

In [6]:
testloader = torch.utils.data.DataLoader(testset, shuffle=True,
                                        batch_size=60)

# Part 2 Design Model

In [7]:
from torch import nn, optim
import torch.nn.functional as F

In [8]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [9]:
model = Model()

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005)

# Part 3: Train the model

In [11]:
print(len(trainloader))

1000


In [12]:
epochs = 30

train_losses, test_losses= [], []

for epoch in range(epochs):
    train_loss = 0
    test_loss = 0
    accuracy = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        op = model(images)
        loss = criterion(op, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        
    with torch.no_grad():
        model.eval()
        for images, labels in testloader:
            log_ps = model(images)
            prob = torch.exp(log_ps)
            top_probs, top_classes = prob.topk(1, dim=1)
            equals = labels == top_classes.view(labels.shape)
            accuracy += equals.type(torch.FloatTensor).mean()
            test_loss += criterion(log_ps, labels)
        model.train()
    print("Epoch: {}/{}...".format(epoch + 1, epochs),
         "Training loss: {:.3f}...".format(train_loss / len(trainloader)),
         "Test loss: {:.3f}...".format(test_loss / len(testloader)),
         "Test Accuracy: {:.3f}".format(accuracy / len(testloader)))
    train_losses.append(train_loss / len(trainloader))
    test_losses.append(test_loss / len(testloader))

Epoch: 1/30... Training loss: 0.617... Test loss: 0.459... Test Accuracy: 0.841
Epoch: 2/30... Training loss: 0.401... Test loss: 0.398... Test Accuracy: 0.861
Epoch: 3/30... Training loss: 0.353... Test loss: 0.357... Test Accuracy: 0.875
Epoch: 4/30... Training loss: 0.325... Test loss: 0.342... Test Accuracy: 0.877
Epoch: 5/30... Training loss: 0.306... Test loss: 0.324... Test Accuracy: 0.884
Epoch: 6/30... Training loss: 0.291... Test loss: 0.325... Test Accuracy: 0.882
Epoch: 7/30... Training loss: 0.280... Test loss: 0.311... Test Accuracy: 0.889
Epoch: 8/30... Training loss: 0.270... Test loss: 0.328... Test Accuracy: 0.879
Epoch: 9/30... Training loss: 0.262... Test loss: 0.309... Test Accuracy: 0.889
Epoch: 10/30... Training loss: 0.254... Test loss: 0.312... Test Accuracy: 0.886
Epoch: 11/30... Training loss: 0.248... Test loss: 0.288... Test Accuracy: 0.895
Epoch: 12/30... Training loss: 0.243... Test loss: 0.286... Test Accuracy: 0.900
Epoch: 13/30... Training loss: 0.237.